In [83]:
import pandas as pd

In [84]:
Export = True

In [85]:
filepath1 = 'data/expend13-20.csv'
first = pd.read_csv(filepath1, header=2)

In [86]:
filepath2 = 'data/expend21-23.csv'
second = pd.read_csv(filepath2, header=2)

In [87]:
def clean_string(string):
    # Handle NaN/None values
    if pd.isna(string) or string is None:
        return string
    
    # Convert to string if not already
    string = str(string)
    
    # Apply existing replacements
    string = string.replace('b/', '')
    string = string.replace('a/', '')
    string = string.replace('*', '')
    string = string.replace('White, Asian, and all other races, not including Black or African-American', 'White, Asian, and all other races')
    
    # New standardizations for naming inconsistencies:
    
    # Standardize "out-of-town" vs "out of town"
    string = string.replace('Food prepared by consumer unit on out of town trips', 
                          'Food prepared by consumer unit on out-of-town trips')
    
    # Standardize appliances naming
    string = string.replace('Small appliances and miscellaneous housewares', 
                          'Small appliances, miscellaneous housewares')
    
    # Standardize gasoline naming (remove trailing annotations)
    string = string.replace('Gasoline and other fuels f/', 'Gasoline, other fuels, and motor oil')
    
    # Standardize stimulus payment descriptions
    string = string.replace('Stimulus payment (through Q20221)', 'Stimulus payment (new UCC Q20202)#')
    
    # Remove new UCC annotations for consistency
    string = string.replace('Laundry and cleaning products (new UCC Q20231)', 'Laundry and cleaning supplies')
    
    # Strip whitespace
    string = string.strip()
    
    return string

def transform_first_df(first):
    years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
    headers = first.T.iloc[0].tolist()
    
    # Clean headers using list comprehension for efficiency
    headers_clean = [clean_string(header) for header in headers]
    
    transposed = first.T[1:]
    transposed.columns = headers_clean
    transposed = transposed.dropna(axis=1, how='all')
    transposed.index = years
    transposed.index.name = "Year"
    transposed = transposed.reset_index()
    return transposed

def transform_second_df(second):
    years = [2021, 2022, 2023]
    headers = second.T.iloc[0].tolist()
    
    # Clean headers using list comprehension for efficiency
    headers_clean = [clean_string(header) for header in headers]
    
    transposed = second.T[1:]
    # Fix: Use headers_clean instead of headers
    transposed.columns = headers_clean
    transposed = transposed.dropna(axis=1, how='all')
    transposed.index = years
    transposed.index.name = "Year"
    transposed = transposed.reset_index() 
    # Fix: Remove the incorrect df.rename line (df was undefined)
    # If you need to rename empty columns, do it properly:
    # transposed.rename(columns={'': 'new_name1'}, inplace=True)
    
    return transposed

In [88]:
first_transformed = transform_first_df(first)
first_transformed.head(5)

,Year,Number of consumer units (in thousands),Income before taxes,Income after taxes,Age of reference person,People,Children under 18,Adults 65 and older,Earners,Vehicles,...,All other apparel products and services,Transportation,Healthcare,Entertainment,"Toys, games, arts and crafts, and tricycles",Other entertainment,Personal care products and services,Reading,Education,All other gifts
0,2013,"125,670","$63,784","56,352",50.1,2.5,.6,.3,1.3,1.9,...,28,100,35,89,27,62,14,5,274,72
1,2014,"127,006","$66,877","58,364",50.3,2.5,.6,.4,1.3,1.9,...,36,108,35,89,28,61,13,4,236,65
2,2015,"128,437","$69,627","60,448",50.5,2.5,.6,.4,1.3,1.9,...,30,139,34,95,29,66,16,6,278,77
3,2016,"129,549","$74,664","64,175",50.9,2.5,.6,.4,1.3,1.9,...,36,101,28,95,43,52,13,4,276,92
4,2017,"130,001","$73,573","63,606",50.9,2.5,.6,.4,1.3,1.9,...,34,141,21,98,39,59,12,3,314,102


In [89]:
second_transformed = transform_second_df(second)
second_transformed.head(5)

,Year,Number of consumer units (in thousands),Income before taxes,Income after taxes,Age of reference person,People,Children under 18,Adults 65 and older,Earners,Vehicles,...,Personal taxes (contains some imputed values),Federal income taxes,Stimulus payment (new UCC Q20202)#,State and local income taxes,Other taxes,Income after taxes,Other money receipts,Mortgage principal paid on owned property,Estimated market value of owned home,Estimated monthly rental value of owned home
0,2021,"133,595","$87,432","78,743",51.8,2.4,.6,.4,1.3,1.9,...,"8,689","8,561","-2,542","2,564",105,"78,743","1,004","-2,621","253,645","1,265"
1,2022,"134,090","$94,003","83,195",52.1,2.4,.6,.4,1.3,1.9,...,"10,809","7,968",0,"2,757",84,"83,195",d/,"-2,843","279,189","1,401"
2,2023,"134,556","$101,805","87,869",52.1,2.5,.6,.4,1.3,1.9,...,"13,937","10,912",e/,"2,942",83,"87,869",d/,"-2,935","292,518","1,500"


In [90]:
# Check the full index of both DataFrames
print("First DataFrame index:", first_transformed.index.tolist())
print("Second DataFrame index:", second_transformed.index.tolist())

# Check for any duplicates
print("First has duplicates:", first_transformed.index.duplicated().any())
print("Second has duplicates:", second_transformed.index.duplicated().any())

# Check data types
print("First index dtype:", first_transformed.index.dtype)
print("Second index dtype:", second_transformed.index.dtype)

First DataFrame index: [0, 1, 2, 3, 4, 5, 6, 7]
Second DataFrame index: [0, 1, 2]
First has duplicates: False
Second has duplicates: False
First index dtype: int64
Second index dtype: int64


In [91]:
# Check the actual shape and info
print("First shape:", first_transformed.shape)
print("Second shape:", second_transformed.shape)
print("First columns:", first_transformed.columns.tolist())
print("Second columns:", second_transformed.columns.tolist())

First shape: (8, 175)
Second shape: (3, 148)
First columns: ['Year', 'Number of consumer units (in thousands)', 'Income before taxes', 'Income after taxes', 'Age of reference person', 'People', 'Children under 18', 'Adults 65 and older', 'Earners', 'Vehicles', 'Men', 'Women', 'Homeowner', 'With mortgage', 'Without mortgage', 'Renter', 'Black or African-American', 'White, Asian, and all other races', 'Hispanic or Latino', 'Not Hispanic or Latino', 'Elementary (1-8)', 'High school (9-12)', 'College', 'Never attended and other', 'At least one vehicle owned or leased', 'Average annual expenditures', 'Food', 'Food at home', 'Cereals and bakery products', 'Cereals and cereal products', 'Bakery products', 'Meats, poultry, fish, and eggs', 'Beef', 'Pork', 'Other meats', 'Poultry', 'Fish and seafood', 'Eggs', 'Dairy products', 'Fresh milk and cream', 'Other dairy products', 'Fruits and vegetables', 'Fresh fruits', 'Fresh vegetables', 'Processed fruits', 'Processed vegetables', 'Other food at ho

In [92]:
def compare_columns(df1_cols, df2_cols):
    set1 = set(df1_cols)
    set2 = set(df2_cols)
    
    print("Columns only in first DataFrame:")
    print(sorted(set1 - set2))
    print(f"\nCount: {len(set1 - set2)}")
    
    print("\nColumns only in second DataFrame:")
    print(sorted(set2 - set1))
    print(f"\nCount: {len(set2 - set1)}")
    
    print("\nCommon columns:")
    print(f"Count: {len(set1 & set2)}")

# Usage:
compare_columns(first_transformed.columns.tolist(), second_transformed.columns.tolist())

Columns only in first DataFrame:
['All other apparel products and services', 'All other gifts', 'Appliances and miscellaneous housewares', 'Females, 2 and over', 'Gifts of good and services, total (thru 2019)', 'Interest, dividends, rental income, other property income', 'Jewelry and watches', 'Maintenance, repairs, insurance, other expenses', 'Males, 2 and over', 'Net change in total assets', 'Net change in total assets and liabilities', 'Net change in total liabilities', 'Other entertainment', 'Other housing', 'Toys, games, arts and crafts, and tricycles']

Count: 15

Columns only in second DataFrame:
['Gasoline', 'Interest, dividends, rental income, and other property income', 'Maintenance, repairs, insurance, and other expenses', 'Processed fruits and vegetables']

Count: 4

Common columns:
Count: 141


In [93]:
first_transformed = first_transformed.set_index("Year")
second_transformed = second_transformed.set_index("Year")
# Check if index names match
print("First index name:", first_transformed.index.name)
print("Second index name:", second_transformed.index.name)
# Check the data types of your indices
print("First DataFrame index dtype:", first_transformed.index.dtype)
print("Second DataFrame index dtype:", second_transformed.index.dtype)

# Check for any non-numeric or mixed types
print("First DataFrame index values:", first_transformed.index.tolist())
print("Second DataFrame index values:", second_transformed.index.tolist())
# Check for NaN values in the index
print("NaN in first index:", first_transformed.index.isna().sum())
print("NaN in second index:", second_transformed.index.isna().sum())

First index name: Year
Second index name: Year
First DataFrame index dtype: int64
Second DataFrame index dtype: int64
First DataFrame index values: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
Second DataFrame index values: [2021, 2022, 2023]
NaN in first index: 0
NaN in second index: 0


In [94]:
# Check for duplicate column names
print("Duplicate columns in first_transformed:")
duplicate_cols_1 = first_transformed.columns[first_transformed.columns.duplicated()]
print(duplicate_cols_1.tolist())
print(f"Count: {len(duplicate_cols_1)}")

print("\nDuplicate columns in second_transformed:")
duplicate_cols_2 = second_transformed.columns[second_transformed.columns.duplicated()]
print(duplicate_cols_2.tolist())
print(f"Count: {len(duplicate_cols_2)}")

# Also check column counts
print(f"\nFirst DF columns: {len(first_transformed.columns)}")
print(f"Second DF columns: {len(second_transformed.columns)}")

Duplicate columns in first_transformed:
['Income before taxes', 'Income after taxes', 'Food', 'Alcoholic beverages', 'Housing', 'Housekeeping supplies', 'Household textiles', 'Major appliances', 'Small appliances, miscellaneous housewares', 'Miscellaneous household equipment', 'Apparel and services', 'Children under 2', 'Other apparel products and services', 'Transportation', 'Healthcare', 'Entertainment', 'Personal care products and services', 'Reading', 'Education']
Count: 19

Duplicate columns in second_transformed:
['Laundry and cleaning supplies', 'Income before taxes', 'Income after taxes']
Count: 3

First DF columns: 174
Second DF columns: 147


In [95]:
first_cleaned = first_transformed.loc[:, ~first_transformed.columns.duplicated(keep='first')]
second_cleaned = second_transformed.loc[:, ~second_transformed.columns.duplicated(keep='first')]
combined_df = pd.concat([first_cleaned, second_cleaned], axis=0, ignore_index=False, sort=False)

In [96]:
for col in first_transformed.columns:
    print(col)

Number of consumer units (in thousands)
Income before taxes
Income after taxes
Age of reference person
People
Children under 18
Adults 65 and older
Earners
Vehicles
Men
Women
Homeowner
With mortgage
Without mortgage
Renter
Black or African-American
White, Asian, and all other races
Hispanic or Latino
Not Hispanic or Latino
Elementary (1-8)
High school (9-12)
College
Never attended and other
At least one vehicle owned or leased
Average annual expenditures
Food
Food at home
Cereals and bakery products
Cereals and cereal products
Bakery products
Meats, poultry, fish, and eggs
Beef
Pork
Other meats
Poultry
Fish and seafood
Eggs
Dairy products
Fresh milk and cream
Other dairy products
Fruits and vegetables
Fresh fruits
Fresh vegetables
Processed fruits
Processed vegetables
Other food at home
Sugar and other sweets
Fats and oils
Miscellaneous foods
Nonalcoholic beverages
Food prepared by consumer unit on out-of-town trips
Food away from home
Alcoholic beverages
Housing
Shelter
Owned dwelling

In [97]:
for col in second_transformed.columns:
    print(col)

Number of consumer units (in thousands)
Income before taxes
Income after taxes
Age of reference person
People
Children under 18
Adults 65 and older
Earners
Vehicles
Men
Women
Homeowner
With mortgage
Without mortgage
Renter
Black or African-American
White, Asian, and all other races
Hispanic or Latino
Not Hispanic or Latino
Elementary (1-8)
High school (9-12)
College
Never attended and other
At least one vehicle owned or leased
Average annual expenditures
Food
Food at home
Cereals and bakery products
Cereals and cereal products
Bakery products
Meats, poultry, fish, and eggs
Beef
Pork
Other meats
Poultry
Fish and seafood
Eggs
Dairy products
Fresh milk and cream
Other dairy products
Fruits and vegetables
Fresh fruits
Fresh vegetables
Processed fruits
Processed vegetables
Processed fruits and vegetables
Other food at home
Sugar and other sweets
Fats and oils
Miscellaneous foods
Nonalcoholic beverages
Food prepared by consumer unit on out-of-town trips
Food away from home
Alcoholic beverage

In [98]:
for col in combined_df.columns:
    print(col)

Number of consumer units (in thousands)
Income before taxes
Income after taxes
Age of reference person
People
Children under 18
Adults 65 and older
Earners
Vehicles
Men
Women
Homeowner
With mortgage
Without mortgage
Renter
Black or African-American
White, Asian, and all other races
Hispanic or Latino
Not Hispanic or Latino
Elementary (1-8)
High school (9-12)
College
Never attended and other
At least one vehicle owned or leased
Average annual expenditures
Food
Food at home
Cereals and bakery products
Cereals and cereal products
Bakery products
Meats, poultry, fish, and eggs
Beef
Pork
Other meats
Poultry
Fish and seafood
Eggs
Dairy products
Fresh milk and cream
Other dairy products
Fruits and vegetables
Fresh fruits
Fresh vegetables
Processed fruits
Processed vegetables
Other food at home
Sugar and other sweets
Fats and oils
Miscellaneous foods
Nonalcoholic beverages
Food prepared by consumer unit on out-of-town trips
Food away from home
Alcoholic beverages
Housing
Shelter
Owned dwelling

### DataFrame Summaries

#### `df2_clean`
- **Type:** `pandas.core.frame.DataFrame`
- **Shape:** (3 rows × 144 columns)
- **Index:** RangeIndex (0 to 2)
- **Columns:** Adults 65 and older, Age of reference person, Alcoholic beverages, ... , Women, 16 and over, Year
- **Dtypes:** int64(1), object(143)
- **Memory Usage:** 3.5+ KB

#### `df2_full`
- **Type:** `pandas.core.frame.DataFrame`
- **Shape:** (3 rows × 162 columns)
- **Index:** Year (2021, 2022, 2023)
- **Columns:** Adults 65 and older, Age of reference person, Alcoholic beverages, ... , Women, 16 and over
- **Dtypes:** object(162)
- **Memory Usage:** 3.9+ KB

#### Duplicate Columns
- **In first_transformed:**  
	`['Income before taxes', 'Income after taxes', 'Food', 'Alcoholic beverages', 'Housing', 'Housekeeping supplies', 'Household textiles', 'Major appliances', 'Small appliances, miscellaneous housewares', 'Miscellaneous household equipment', 'Apparel and services', 'Children under 2', 'Other apparel products and services', 'Transportation', 'Healthcare', 'Entertainment', 'Personal care products and services', 'Reading', 'Education']`
- **In second_transformed:**  
	`['Laundry and cleaning supplies', 'Income before taxes', 'Income after taxes']`

#### Example Column Names
- `example1`: 'Income after taxes*'
- `example2`: 'Income after taxes b/'

#### Filepaths
- `filepath1`: `'data/expend13-20.csv'`
- `filepath2`: `'data/expend21-23.csv'`

#### DataFrames
- **first**: 233 rows × 9 columns (raw 2013–2020 data)
- **first_cleaned**: 8 rows × 155 columns (cleaned, indexed by year)
- **first_transformed**: 8 rows × 174 columns (transformed, indexed by year)
- **second**: 202 rows × 4 columns (raw 2021–2023 data)
- **second_cleaned**: 3 rows × 144 columns (cleaned, indexed by year)
- **second_transformed**: 3 rows × 147 columns (transformed, indexed by year)

---

**Note:**  
- All DataFrames are indexed by year except for `df2_clean` (default RangeIndex).
- Most columns are of type `object` due to mixed data (numbers, strings, missing values).
- Duplicate columns exist due to inconsistent naming conventions across years.

In [99]:
if Export:
    combined_df.to_csv('data/combined_df.csv')
